Kendi train_test_split fonksiyonumu oluşturma sebebim, verinin zaman serisinden oluşması. Eğer train_test_split'i kullansaydım rastgele bir şekilde veriyi dağıtıp, zaman serisini ve ardışıklığı bozacaktı.

In [ ]:
def train_test_split(train,percent = 0.8):
  n_rows = train.shape[0]
  df_train = train.iloc[:int(percent*n_rows), :]
  df_test = train.iloc[int(percent*n_rows):, :]
  X_train = df_train.drop("d_enerji",axis = 1)
  y_train = df_train["d_enerji"]
  X_test = df_test.drop("d_enerji",axis = 1)
  y_test = df_test["d_enerji"] #y_true
  return X_train, y_train, X_test, y_test

Polynomial Regression

In [ ]:
def polynomialRegression(train,sub_df):
  test_dict = hyperParameterOptimizationForPolynomialRegression(train)
  min_key = min(test_dict.keys())
  print("MAPE (Mean Absolute Percentage Error)= ", min_key)
  best_parameters = test_dict.get(min_key)
  degree = best_parameters["degree"]
  print(degree)
  import matplotlib.pyplot as plt
  from sklearn.linear_model import LinearRegression
  from sklearn.preprocessing import PolynomialFeatures
  X = train.drop("d_enerji",axis = 1)
  y = train["d_enerji"]

  poly = PolynomialFeatures(degree = degree)
  x_poly = poly.fit_transform(X)
  lin_reg = LinearRegression()
  lin_reg.fit(x_poly,y)

  x_sub = sub_df.drop("d_enerji", axis=1)
  x_sub_poly = poly.transform(x_sub)
  y_pred = lin_reg.predict(x_sub_poly)
  return y_pred

def hyperParameterOptimizationForPolynomialRegression(train):
  X_train, y_train, X_test, y_test = train_test_split(train)
  from sklearn.linear_model import LinearRegression
  from sklearn.preprocessing import PolynomialFeatures
  degree_list = [2,3]
  results_dict = {}
  for degree in degree_list:
    poly = PolynomialFeatures(degree = degree)
    x_poly = poly.fit_transform(X_train)
    lin_reg = LinearRegression()
    lin_reg.fit(x_poly,y_train)
    x_test_poly = poly.transform(X_test)
    y_pred = lin_reg.predict(x_test_poly)

    mape_result = mape(y_test,y_pred)
    temp_dict = { mape_result: {"degree" : degree} }
    results_dict.update(temp_dict)

  return results_dict

RandomForestRegressor

In [ ]:
def hyperParameterOptimizationForRandomForestRegressor(train):
  from sklearn.ensemble import RandomForestRegressor
  from sklearn.metrics import mean_squared_error
  X_train, y_train, X_test, y_test = train_test_split(train)

  n_estimators_list = [100,250,500] # Karar ağacı sayısı. Genellikle 100-500 arasında bir değer seçilir.
  max_features_list = [] # max_features: Ağacın dallanması için göz önünde bulundurulacak özellik sayısı. Bu parametre için genellikle sqrt (karekök), log2 (logaritma tabanı 2) veya bir sabit değer kullanılır.
  # max_depth_list = [] # Karar ağacının maksimum derinliği. Bu parametre için genellikle None (sınırsız derinlik), 10 veya 20 gibi bir değer kullanılır.
  min_samples_split_list = [2,3,5] # min_samples_split: Dallanmanın gerçekleşmesi için minimum örnek sayısı. Bu parametre için genellikle 2-5 arasında bir değer kullanılır.
  min_samples_leaf_list = [2,3,4] # min_samples_leaf: Bir yaprağın oluşması için minimum örnek sayısı. Bu parametre için genellikle 1-5 arasında bir değer kullanılır.
  results_dict = {}
  for n_estimators in n_estimators_list:
    for min_samples_split in min_samples_split_list:
      for min_samples_leaf in min_samples_leaf_list:
        rf_reg = RandomForestRegressor(n_estimators=n_estimators, min_samples_split = min_samples_split, min_samples_leaf = min_samples_leaf,random_state=42)
        rf_reg.fit(X_train, y_train)
        y_pred = rf_reg.predict(X_test)
        mape_result = mape(y_test,y_pred)
        temp_dict = { mape_result: {"n_estimators" : n_estimators,"min_samples_split" : min_samples_split,"min_samples_leaf" : min_samples_leaf} }
        results_dict.update(temp_dict)
  return results_dict

def RandomForestRegressor(train,sub_df):
  from sklearn.ensemble import RandomForestRegressor
  X = train.drop("d_enerji",axis = 1)
  y = train["d_enerji"]
  test_dict = hyperParameterOptimizationForRandomForestRegressor(train)
  min_key = min(test_dict.keys())
  print("MAPE (Mean Absolute Percentage Error)= ", min_key)
  best_parameters = test_dict.get(min_key)
  n_estimators = best_parameters["n_estimators"]
  min_samples_split = best_parameters["min_samples_split"]
  min_samples_leaf = best_parameters["min_samples_leaf"]
  print(n_estimators,min_samples_split,min_samples_leaf)
  rf_reg = RandomForestRegressor(n_estimators= n_estimators, min_samples_split = min_samples_split, min_samples_leaf = min_samples_leaf,random_state=42)
  rf_reg.fit(X, y)
  y_pred = rf_reg.predict(sub_df.drop("d_enerji",axis=1))
  return y_pred

GradientBoostRegressor

In [ ]:
def hyperParameterOptimizationForGradientBoostingRegressor(train):
  from sklearn.ensemble import GradientBoostingRegressor
  from sklearn.metrics import mean_squared_error
  X_train, y_train, X_test, y_test = train_test_split(train)
  n_estimators_list = [250,500] # Karar ağacı sayısı. Genellikle 100-500 arasında bir değer seçilir.
  learning_rate_list = [0.1, 0.01, 0.05]
  max_depth_list = [3,4,5] #  Her ağacın maksimum derinliğini belirler. Daha derin ağaçlar, daha karmaşık modeller oluşturur ancak aynı zamanda aşırı uyuma neden olabilir. Genellikle, 3-5 arası değerler kullanılır.
  min_samples_split_list = [5,7] # Her iç düğümün ayrılması için minimum örnek sayısını belirler. Daha yüksek bir değer, daha basit modeller oluşturur ancak aynı zamanda daha az esneklik sağlar. Genellikle, 2-10 arası değerler kullanılır.
  # subsample_list = [0.5,0.75] # subsample: Eğitim örneklerinin alt kümesini seçmenizi sağlar. Bu, ağaçlar arasındaki çeşitliliği artırır ve aşırı uyum riskini azaltabilir. Genellikle, 0.5-1 arası değerler kullanılır.
  results_dict = {}
  for n_estimators in n_estimators_list:
    for learning_rate in learning_rate_list:
      for max_depth in max_depth_list:
        for min_samples_split in min_samples_split_list:
          rf_reg = GradientBoostingRegressor(n_estimators=n_estimators, min_samples_split = min_samples_split, learning_rate = learning_rate, max_depth = max_depth)
          rf_reg.fit(X_train, y_train)
          y_pred = rf_reg.predict(X_test)
          mape_result = mape(y_test,y_pred)
          temp_dict = { mape_result: {"n_estimators": n_estimators, "min_samples_split" : min_samples_split, "learning_rate" : learning_rate, "max_depth" : max_depth} }
          results_dict.update(temp_dict)
  return results_dict

def GradientBoostingRegressor(train,sub_df):
  from sklearn.ensemble import GradientBoostingRegressor
  X = train.drop("d_enerji",axis = 1)
  y = train["d_enerji"]
  test_dict = hyperParameterOptimizationForGradientBoostingRegressor(train)
  min_key = min(test_dict.keys())
  print("MAPE (Mean Absolute Percentage Error)= ", min_key)
  best_parameters = test_dict.get(min_key)
  n_estimators = best_parameters["n_estimators"]
  min_samples_split = best_parameters["min_samples_split"]
  learning_rate = best_parameters["learning_rate"]
  max_depth = best_parameters["max_depth"]
  print(n_estimators,min_samples_split,learning_rate,max_depth)
  gbr = GradientBoostingRegressor(n_estimators=n_estimators,min_samples_split=min_samples_split, learning_rate=learning_rate, max_depth=max_depth)
  gbr.fit(X, y)
  y_pred = gbr.predict(sub_df.drop("d_enerji",axis=1))
  return y_pred

XGBoost

In [ ]:
def mape(y_true, y_pred): # MAPE (Mean Absolute Percentage Error)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def hyperParameterOptimizationForXGBoost(train):
  X_train, y_train, X_test, y_test = train_test_split(train)
  from xgboost import XGBRegressor
  max_depth_list = [3,5,7] # Karar ağacıların maksimum derinliği. 3-10 arası bir değer genellikle önerilir.
  learning_rate_list = [0.1, 0.01, 0.05] # Her ağaç oluşturma işleminde kullanılan öğrenme oranı. 0.01-0.1 arası bir değer genellikle önerilir.
  n_estimators_list = [100,250,500,1000] # Oluşturulacak ağaç sayısı. 50-500 arası bir değer genellikle önerilir. 50,100,250,500,1000
  results_dict = {}
  for max_depth in max_depth_list:
    for learning_rate in learning_rate_list:
      for n_estimator in n_estimators_list:
        xgb_model = XGBRegressor(max_depth = max_depth, n_estimators = n_estimator, learning_rate= learning_rate)
        xgb_model.fit(X_train,y_train)
        y_pred = xgb_model.predict(X_test)
        mape_result = mape(y_test,y_pred)
        temp_dict = { mape_result: {"max_depth" : max_depth,"learning_rate" : learning_rate,"n_estimators" : n_estimator}}
        results_dict.update(temp_dict)
  return results_dict

def bestParamsForXGBoost(train):
  test_dict = hyperParameterOptimizationForXGBoost(train)
  min_key = min(test_dict.keys())
  print("MAPE (Mean Absolute Percentage Error)= ", min_key)
  best_parameters = test_dict.get(min_key)
  max_depth = best_parameters["max_depth"]
  learning_rate = best_parameters["learning_rate"]
  n_estimators = best_parameters["n_estimators"]
  print(f"max_depth ={max_depth}, learning_rate = {learning_rate}, n_estimators = {n_estimators}")
  return max_depth,learning_rate,n_estimators

def gridSearchCVForXGBoost(train):
  from sklearn.model_selection import GridSearchCV
  from xgboost import XGBRegressor
  from sklearn.metrics import make_scorer
  from sklearn.metrics import mean_absolute_percentage_error
  X_train, y_train, X_test, y_test = train_test_split(train)
  scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)
  model = XGBRegressor()
  params = {
    'n_estimators': [100,250,500,1000],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.05]
  }
  grid = GridSearchCV(
      estimator = model, 
      param_grid = params, 
      scoring=scorer,
      n_jobs=-1,
      verbose=3,
      cv=5)
  grid.fit(X_train, y_train)
  thebestresult = grid.best_score_
  print(thebestresult)
  print("En iyi parametreler:", grid.best_params_)
  # best_model = grid.best_estimator_
  # best_model.fit(X_train, y_train)

def XGBoost(train_data,sub_df):
  from xgboost import XGBRegressor
  max_depth, learning_rate, n_estimators = bestParamsForXGBoost(train)
  xgb_model = XGBRegressor(max_depth = max_depth, n_estimators = n_estimators, learning_rate= learning_rate)
  xgb_model.fit(train_data.drop("d_enerji",axis = 1),train_data["d_enerji"])
  y_pred = xgb_model.predict(sub_df.drop("d_enerji",axis = 1))
  return y_pred

sarimax

p = 1-3, d = 0-2, q = 1-3: Bu, otoregresif (AR), fark (I) ve hareketli ortalama (MA) terimlerinin sayısı için tipik bir aralıktır.

P = 0-3, D = 0-1, Q = 0-3, s = 12: Bu, sezonluk AR, fark ve hareketli ortalama terimleri için tipik bir aralıktır. s, verilerin sezonluk bir desen içerip içermediğine bağlı olarak değişebilir.

exog: Eğer SARIMAX modelinde eksojen değişkenler varsa, bunların sayısı ve türü de modelin amacına bağlı olarak değişebilir.

In [ ]:
def sarimax():
  from statsmodels.tsa.statespace.sarimax import SARIMAX
  # model = SARIMAX( endog = train["d_enerji"] , order=(p, d, q), seasonal_order=(P, D, Q, S))
  model = SARIMAX( endog = train["d_enerji"] , order=(1, 1, 1), seasonal_order=(1, 1, 1, 12)) # P = 
  result = model.fit()
  predictions = result.predict(start = sub.index[0], end = sub.index[-1])
  return predictions

In [ ]:
def prophet(train,sub_df,holidays):
  train_for_model = train.copy()
  train_for_model = train_for_model.rename(columns = {
      "tarihwithhours":"ds",
      "d_enerji":"y"
  })
  sub_for_model = sub_df.copy()
  sub_for_model = sub_for_model.rename(columns = {"tarihwithhours":"ds"})

  from prophet import Prophet # 5.54701, hiçbir seasonality eklemeden.
  model = Prophet(holidays = holidays)
  model.add_seasonality(name='weekly', period=7, fourier_order=5, condition_name='is_weekend')
  model.add_seasonality(name='yearly', period=365.25/4, fourier_order=10, condition_name='is_spring')
  model.add_seasonality(name='yearly', period=365.25/4, fourier_order=10, condition_name='is_summer')
  model.add_seasonality(name='yearly', period=365.25/4, fourier_order=10, condition_name='is_fall')
  model.add_seasonality(name='yearly', period=365.25/4, fourier_order=10, condition_name='is_winter')
  model.add_seasonality(name='monthly', period=30.5, fourier_order=5, mode='additive') #
  model.add_seasonality(name='yearly', period=365.25, fourier_order=10, mode='additive')
  model.add_seasonality(name='hourly', period=24, fourier_order=5)

  model.fit(train_for_model)

  forecast = model.predict(sub_for_model)
  y_pred = forecast["yhat"]
  return y_pred